In [1]:
from src.helper import load_matrix
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix

In [2]:
BASE_PATH = 'D:/Coding/Project/chainuser'

In [33]:
def get_time(sec):
    day = sec // (24*3600)
    sec = sec % (24*3600)
    hour = sec // 3600
    sec = sec % 3600
    minute = sec // 60
    sec = sec % 60
    day, hour, minute, sec = (map(int, (day, hour, minute, sec)))
    return {'day': day, 'hour': hour, 'minute':minute, 'second':sec}

In [3]:
vaksinasi_df = pd.read_pickle(f'{BASE_PATH}/data/vaksinasi_simplified.pkl')
user_df = pd.read_pickle(f'{BASE_PATH}/data/process.pkl')

In [4]:
user_ser = pd.Series(user_df.index, index=user_df['indice'], name='name')
user_ser.index.set_names(None, inplace=True)

In [5]:
lil = load_matrix(f'{BASE_PATH}\data\chainmatrix.npz')
csc = lil.tocsc()
csr = lil.tocsr()

In [6]:
nz_rows, nz_cols = lil.nonzero()

In [7]:
id_mr_x = 10073
mr_x = user_ser[id_mr_x]
mr_x

'diskominfo_wsb'

In [8]:
followers_1, _ = csc.getcol(id_mr_x).nonzero()  # followers_1 adalah followernya mr_x
followers_1

array([ 1958,  4091, 19602, 20733, 20935, 27873])

In [13]:
csc[followers_1, id_mr_x].toarray().flatten() 

array([ 75245., 193293., 148415., 169703., 194587.,  64460.],
      dtype=float32)

In [14]:
followers_1 = nz_rows[np.isin(nz_cols, id_mr_x)]
followers_1

array([ 1958,  4091, 19602, 20733, 20935, 27873])

In [15]:
followers_1[np.isin(followers_1, nz_cols)]  # beberapa followersnya mr x punya follower juga

array([ 4091, 19602, 20733])

In [17]:
followers_2 = nz_rows[np.isin(nz_cols, followers_1)]  # orang-orang ini memfollow followersnya mr x
followers_2

array([ 1590,  5990, 17541, 33579])

In [18]:
followers_2[np.isin(followers_2, nz_cols)]

array([ 1590,  5990, 33579])

In [19]:
followers_3 = nz_rows[np.isin(nz_cols, followers_2)]  # orang-orang ini memfollow followersnya followers mr x
followers_3

array([  417,  3137,  3531, 11097, 14654, 20294, 26357, 26970, 29470,
       32610, 33400, 33654, 33655])

In [21]:
followers_group = []
current_cols = np.asarray([id_mr_x])
# followers_group.append(current_cols)
# print(current_cols.shape[0])

ct = 0
thres = 2
while True:
    # popular_followers = np.isin(nz_cols, current_cols)  # follower yg difollow orang
    # followers = nz_rows[popular_followers]  # followers dari para follower yang populer
    followers, cols = csc[:, current_cols].nonzero()
    popular_followers = current_cols[cols]
    # followers_group.append(np.array([followers, popular_followers]))
    print(np.array([followers, popular_followers]))
    # print(popular_followers)
    # print(popular_followers.shape[0])
    current_cols = followers
    
    ct += 1
    if ct >= thres:
        break


[[ 1958  4091 19602 20733 20935 27873]
 [10073 10073 10073 10073 10073 10073]]
[[ 1590  5990 17541 33579]
 [ 4091 19602 20733 20733]]


In [34]:
get_time(lil[4091, 10073])

{'day': 2, 'hour': 5, 'minute': 41, 'second': 33}

In [22]:
user_ser[1590]

'AhYanuar'

In [35]:
influence_matrix = lil_matrix((1, lil.shape[0]))
previous_followers = np.array([id_mr_x])

while True:
    followers, cols = csc[:, previous_followers].nonzero()
    if not followers.shape[0]:
        break

    popular_followers = previous_followers[cols]

    for i, j in zip(followers, popular_followers):
        current_value = influence_matrix[0, i]
        previous_follower_value = influence_matrix[0, j]
        target = csr[i, j] + previous_follower_value
        if current_value > 0:
            target = np.minimum(current_value, target)
        influence_matrix[0, i] = target

    previous_followers = followers

In [36]:
_, cols = influence_matrix.nonzero()
res = influence_matrix[0, cols].toarray().flatten()
# print(np.vstack((cols, res)))
sort_args = res.argsort()
res = res[sort_args]
cols = cols[sort_args]

In [37]:
influenced_people = user_ser.loc[cols]
df = pd.DataFrame([res], columns=list(influenced_people), index=pd.Index([mr_x], name='Influencer'))
df.to_csv(f'{BASE_PATH}/data/influenza.csv')

In [39]:
user_df.loc['Banjarnegara__']

indice           5990.0
processed_at    81105.0
Name: Banjarnegara__, dtype: float64